In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

train_dir = "train"
validation_dir = "val"

train_datagen = ImageDataGenerator(
    rescale=1./255,  
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  
    batch_size=32,
    class_mode='categorical'  
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

2024-03-30 23:33:12.538251: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-30 23:33:12.538308: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-30 23:33:12.539953: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-30 23:33:12.548812: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-30 23:33:14.188501: W tensorflow/compiler/tf2

Found 21437 images belonging to 10 classes.
Found 5272 images belonging to 10 classes.


In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2024-03-25 01:19:49.881545: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 01:19:49.881777: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 01:19:49.910895: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-25 01:19:49.981686: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 01:19:51.679340: W tensorflow/compiler/tf2

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

2024-03-25 01:19:54.830203: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-25 01:19:54.830744: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


input_shape = (224, 224, 3)  

model2 = Sequential()

base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers[-5]:
    layer.trainable = False
model2.add(base_model)


model2.add(GlobalAveragePooling2D())
model2.add(Dense(1024, activation='relu'))  
model2.add(Dense(512, activation='relu'))  
model2.add(Dense(128, activation='relu'))  
model2.add(Dense(10, activation='softmax'))  

model2.summary()


2024-03-25 01:30:30.862464: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-25 01:30:30.866067: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-25 01:30:30.866398: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

219055592/219055592 [==============================] - 53s 0us/step


TypeError: 'Conv2D' object is not iterable

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

base_model = DenseNet121(weights='imagenet', include_top=False)
for layer in base_model.layers[:-5]:
    layer.trainable = False

model = Sequential()

model.add(base_model)

model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))  # Assuming 10 classes


model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()],)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, None, None, 1024   7037504   
                             )                                   
                                                                 
 global_average_pooling2d_1  (None, 1024)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             (None, 128)               131200    
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 7169994 (27.35 MB)
Trainable params: 171402 (669.54 KB)
Non-trainable params: 6998592 (26.70 MB)
_________________________________________________________________


In [7]:
model.fit(train_generator,
          epochs=20,
          validation_data=validation_generator)
try:
    model.save("mymod.h5")
except:
    model.save("/home/virendrasinh10/newmod.h5")

Epoch 1/20
670/670 [==============================] - 1417s 2s/step - loss: 1.4615 - accuracy: 0.4785 - precision: 0.7361 - recall: 0.2663 - val_loss: 1.3578 - val_accuracy: 0.5135 - val_precision: 0.6944 - val_recall: 0.3557
Epoch 2/20
670/670 [==============================] - 1285s 2s/step - loss: 1.2745 - accuracy: 0.5370 - precision: 0.7454 - recall: 0.3493 - val_loss: 1.2041 - val_accuracy: 0.5679 - val_precision: 0.7377 - val_recall: 0.3911
Epoch 3/20
670/670 [==============================] - 1275s 2s/step - loss: 1.1817 - accuracy: 0.5687 - precision: 0.7557 - recall: 0.3911 - val_loss: 1.2196 - val_accuracy: 0.5597 - val_precision: 0.7199 - val_recall: 0.4270
Epoch 4/20
670/670 [==============================] - 1280s 2s/step - loss: 1.1192 - accuracy: 0.5976 - precision: 0.7660 - recall: 0.4240 - val_loss: 1.1429 - val_accuracy: 0.5880 - val_precision: 0.7483 - val_recall: 0.4427
Epoch 5/20
670/670 [==============================] - 1284s 2s/step - loss: 1.0686 - accuracy: 0

/home/virendrasinh10/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

input_shape = (224, 224, 3)

model = Sequential()

base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=input_shape)
for layer in base_model.layers[:-7]:
    layer.trainable = False
model.add(base_model)

model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, 7, 7, 1664)        12642880  
                                                                 
 global_average_pooling2d (  (None, 1664)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              1704960   
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 128)               65664     
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                      

In [5]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()],)

In [6]:
model.fit(train_generator,
          epochs=40,
          validation_data=validation_generator)
try:
    model.save("densenet.h5")
except:
    model.save("/home/virendrasinh10/newmod.h5")

Epoch 1/40


2024-03-30 01:01:21.098512: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2024-03-30 01:01:33.490291: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2024-03-30 01:01:33.670768: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-30 01:01:33.778175: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-30 01:01:33.901718: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-30 01:01:35.026910: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2024-03-30 01:01:36.124490: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memor

  4/670 [..............................] - ETA: 8:01 - loss: 2.4768 - accuracy: 0.2109 - precision: 0.8333 - recall: 0.0781         

2024-03-30 01:01:48.490987: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.


670/670 [==============================] - 1399s 2s/step - loss: 1.4027 - accuracy: 0.4937 - precision: 0.7384 - recall: 0.2999 - val_loss: 1.1680 - val_accuracy: 0.5833 - val_precision: 0.7664 - val_recall: 0.3964
Epoch 2/40
670/670 [==============================] - 1362s 2s/step - loss: 1.1096 - accuracy: 0.5953 - precision: 0.7530 - recall: 0.4425 - val_loss: 1.1678 - val_accuracy: 0.5867 - val_precision: 0.7054 - val_recall: 0.4723
Epoch 3/40
670/670 [==============================] - 1363s 2s/step - loss: 0.9702 - accuracy: 0.6484 - precision: 0.7774 - recall: 0.5184 - val_loss: 0.9317 - val_accuracy: 0.6737 - val_precision: 0.7792 - val_recall: 0.5802
Epoch 4/40
670/670 [==============================] - 1356s 2s/step - loss: 0.8724 - accuracy: 0.6796 - precision: 0.7878 - recall: 0.5683 - val_loss: 0.9795 - val_accuracy: 0.6538 - val_precision: 0.7484 - val_recall: 0.5641
Epoch 5/40
670/670 [==============================] - 1548s 2s/step - loss: 0.7883 - accuracy: 0.7112 - pre

/home/virendrasinh10/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2

input_shape = (224, 224, 3)

model = Sequential()

base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=input_shape)
for layer in base_model.layers[:-5]:
    layer.trainable = False
model.add(base_model)

model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))

model.summary()

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


adam_optimizer = Adam()


model.compile(optimizer=adam_optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])


lr_scheduler = LearningRateScheduler(lr_schedule)


model.fit(train_generator,
          epochs=40,
          validation_data=validation_generator,
          callbacks=[lr_scheduler])

try:
    model.save("densenet.h5")
except:
    model.save("/home/virendrasinh10/newmod.h5")


Num GPUs Available:  1
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, 7, 7, 1664)        12642880  
                                                                 
 global_average_pooling2d_1  (None, 1664)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 1024)              1704960   
                                                                 
 batch_normalization_3 (Bat  (None, 1024)              4096      
 chNormalization)                                                
                                                                 
 dense_5 (Dense)             (None, 512)               524800    
                                                                 
 batch_normalization_4 (Bat  (N

/home/virendrasinh10/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:

import tensorflow as tf
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2

input_shape = (224, 224, 3)

model = Sequential()

base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=input_shape)
for layer in base_model.layers[:-6]:
    layer.trainable = False
model.add(base_model)

model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))

model.summary()

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

adam_optimizer = Adam()

model.compile(optimizer=adam_optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

lr_scheduler = LearningRateScheduler(lr_schedule)

model.fit(train_generator,
          epochs=40,
          validation_data=validation_generator,
          callbacks=[lr_scheduler])

try:
    model.save("densenet_approach2.h5")
except:
    model.save("/home/virendrasinh10/densenet_approach2.h5")


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, 7, 7, 1664)        12642880  
                                                                 
 global_average_pooling2d_2  (None, 1664)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_8 (Dense)             (None, 1024)              1704960   
                                                                 
 batch_normalization_6 (Bat  (None, 1024)              4096      
 chNormalization)                                                
                                                                 
 dense_9 (Dense)             (None, 512)               524800    
                                                                 
 batch_normalization_7 (Bat  (None, 512)              

2024-03-30 23:27:52.853822: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2024-03-30 23:28:03.773583: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2024-03-30 23:28:03.887630: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-30 23:28:03.916441: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-30 23:28:04.161050: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-30 23:28:04.561137: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2024-03-30 23:28:05.876065: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memor

  4/670 [..............................] - ETA: 5:35 - loss: 4.9562 - accuracy: 0.2109 - precision_2: 0.4000 - recall_2: 0.1406         

2024-03-30 23:28:18.571387: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.


 38/670 [>.............................] - ETA: 16:12 - loss: 3.8634 - accuracy: 0.3289 - precision_2: 0.5337 - recall_2: 0.2146

KeyboardInterrupt: 

: 